# Neural Network from Scratch

In this lab we'll create a neural network from scratch. In the process we'll learn about the algorithm that makes it all possible, **backpropagation**. In the process we'll create a library **miniflow** and use it to train a neural network on [MNIST](http://yann.lecun.com/exdb/mnist/), a dataset of handwritten digits.

## Intro

TODO: Address the following

* Why am I creating a neural network from scratch?
* Can you give me some motivation?
* Why not just use a library?
* What is your goal for me at the end of this? What should I be able to do that I wasn’t able to do before?

## Functions, Backpropagation and the Chain Rule

We can think of a neural network as a composition of functions with a **loss** (number) as the output. This number can also be thought of as an *error term*, the larger the value the larger our error. Our goal is to get the error as close to 0 as possible.

$$
L = f_n(f_{n-1}(...f_1(x)...))
$$

Now, out of the box a neural network isn't all that useful. Its predictions will at this point will just be noise. We need to be able to train the **neural network to produce the correct output**. 

How do we do this?

Well, what do we have control of? Our inputs? Nope. Our inputs are the data which itself can vary significantly, but we could also change our dataset entirely. We need something that sticks around, something stateful that also affects the output. 

What about our weights? 

Yes, That's exactly it! We can change our weights. They stay static unless we change them. Alright, now we have another question.

How we do alter our weights?

Intuitively we'd like to see what effect a weight had on the output and make a change based on that. If it played a large role in the error then we should alter it by a large amount. If it didn't play a significant role in the error, then it's probably already set to a good value and we should, for the most part, let it be.

How can we formalize this idea?

Luckily for us, this idea is already well established in mathematical literature where it's known as **reverse-mode differentiation** or **backpropagation**. The name sounds very scary but it's actually not so bad. It essentially boils down to computing derivatives and using the chain rule. For example, say we wanted to compute the gradient of the loss function $L$ with respect a weight $W_{jk}$, which is an input to $f_i$. Here $W$ is a matrix of weights so $W_{jk}$ is an individual weight (number) at row $j$, column $k$ of $W$.

$$
L = f_n(f_{n-1}(...f_1(x)...))
$$

Using the [chain rule](https://www.khanacademy.org/math/ap-calculus-ab/product-quotient-chain-rules-ab/chain-rule-ab/a/chain-rule-overview) the computation is

$$
\frac{\partial L}{\partial W_{jk}} = 
\frac{\partial L}{\partial f_n}
\frac{\partial f_n}{\partial f_n}
\frac{\partial f_n}{\partial f_{n-1}}
...
\frac{\partial f_{i+1}}{\partial f_i}
\frac{\partial f_i}{\partial W_{jk}}
$$

#### Quick Aside: Notation

You may have noticed in the above link they used the following notation

$$
\frac {d}{dx} f(g(x)) = f'(g(x))g'(x)
$$

This is equivalent to

$$
\frac {\partial f}{\partial x} = 
\frac {\partial f}{\partial g} \frac {\partial g} {\partial x}
$$

The latter notation is used when we talk about *partial derivatives*, that is, we want measure the effect a specific input had on our output. This is precisely what we're going to be doing throughout this lab so we'll use the latter notation. By the way, the $\partial$ symbol is called a *partial*. 

Ok, now back to our our friend $\frac{\partial L}{\partial W_{jk}}$. This is mathematical speak for "what effect did weight $W_{jk}$ have on the output $L$?". Now you might be wondering why we can't just write

$$
\frac{\partial L}{\partial W_{jk}} = 
\frac{\partial L}{\partial f_i}
\frac{\partial f_i}{\partial W_{jk}}
$$

since $W_{jk}$ is the input to $f_i$. Why do we need to know all these other partials?. If we did that we would miss out on a wealth of information and most likely get an incorrect gradient. Remember that $f_i$ affects $f_{i+1}$ and $f_{i+1}$ affects $f_{i+2}$ and ... you get the point. Missing out on how all these intermediate function affect eachother will won't do us any good and it'll only get worse the more intermediate functions we have.

I lied. We can write

$$
\frac{\partial L}{\partial W_{jk}} = 
\frac{\partial L}{\partial f_i}
\frac{\partial f_i}{\partial W_{jk}}
$$

but

$$
\frac{\partial L}{\partial f_i} =
\frac{\partial L}{\partial f_n}
\frac{\partial f_n}{\partial f_n}
\frac{\partial f_n}{\partial f_{n-1}}
...
\frac{\partial f_{i+1}}{\partial f_i}
$$

otherwise we'll have a bad time!

Ok, that's a lot of abstract thinking let's now take a look at a concrete example.

### Case study: $f = (x * y) + (x * z)$



$$
f = * \hspace{0.5in} f(x,y) = xy
$$

$$
\frac{\partial f}{\partial x} = y \hspace{0.5in} \frac{\partial f}{\partial y} = x
$$

Let's think about this for a bit. What we're saying here is the change of $f$ with respect to $x$ is $y$ and vice-versa. Remember, if our derivative is with respect to $x$ we treat $y$ as a constant. So let's say $y = 10$, think about changing $x$ from 3 to 4. Then $f(3,10) = 30$ and $f(4, 10) = 40$. That's a change in 10 or $y$! Every time we change $x$ by 1, $f$ changes by $y$.


$$
f = + \hspace{0.5in} f(x,y) = x + y
$$

$$
\frac{\partial f}{\partial x} = 1 \hspace{0.5in} \frac{\partial f}{\partial y} = 1
$$

Again, let's think about this. The change in $f$ with respect to $x$ is 1 and same for $y$. This means every time we change $x$ by 1, $f$ will also change by 1. This also shows $x$ and $y$ are independent of eachother.

Ok, let's now use this for a simple function.

TODO: picture here

In [ ]:
# f(x, y, z) = x * y + (x * z)
# we can split these into subexpressions
# g(x, y) = x * y
# h(x, z) = x * z
# f(x, y, z) = g(x, y) + h(x, z)

# intial values
x = 3
y = 4
z = -5

g = x * y
h = x * z

f = g + h
print(f)

Let's take our function $f$ and apply the chain rule to compute the derivatives for $x, y, z$.

$$
\frac{\partial f}{\partial g} = 1 \hspace{0.1in}
\frac{\partial f}{\partial h} = 1 \hspace{0.1in}
\frac{\partial g}{\partial x} = y \hspace{0.1in}
\frac{\partial g}{\partial y} = x \hspace{0.1in}
\frac{\partial h}{\partial x} = z \hspace{0.1in}
\frac{\partial h}{\partial z} = x
$$


$$
\frac{\partial f}{\partial x} = 
\frac{\partial f}{\partial g}
\frac{\partial g}{\partial x}
+
\frac{\partial f}{\partial h}
\frac{\partial h}{\partial x}
\hspace{0.5in}
\frac{\partial f}{\partial y} = 
\frac{\partial f}{\partial g}
\frac{\partial g}{\partial y}
\hspace{0.5in}
\frac{\partial f}{\partial z} = 
\frac{\partial f}{\partial h}
\frac{\partial h}{\partial z}
$$


$$
\frac{\partial f}{\partial x} = 1 * y + 1 * z = y + z
\hspace{0.5in}
\frac{\partial f}{\partial y} = 1 * x = x
\hspace{0.5in}
\frac{\partial f}{\partial z} = 1 * x = x
$$

In [ ]:
# The above in code
x = 3
y = 4
z = -5

dfdg = 1.0
dfdh = 1.0
dgdx = y
dgdy = x
dhdx = z
dhdz = x

dfdx = dfdg * dgdx + dfdh * dhdx
dfdy = dfdg * dgdy
dfdz = dfdh * dhdz

# the output of backpropagation is the gradient
gradient = [dfdx, dfdy, dfdz]
print(gradient) # should be [-1, 3, 3]

Notice the following expression, specifically the `+` function:

$$
\frac{\partial f}{\partial x} = 
\frac{\partial f}{\partial g}
\frac{\partial g}{\partial x}
+
\frac{\partial f}{\partial h}
\frac{\partial h}{\partial x}
$$

Think about how $x,y,z$ flow through the graph. Both $y$ and $z$ have 1 output edge and they follow 1 path to the $f$. On the other hand, $x$ has 2 output edges and follows 2 paths to `f`. Remember, we're calculating **the derivative of f with respect to x**. In order to do this we have to consider all the ways $x$ affects $f$ (all the paths in the graph from $x$ to $f$). 

An easy way to see how many paths we have to consider for a node's derivative is to trace all the paths back from the output node to the input node. So if $f$ is the output node and $x$ is the input node, trace all the paths back from $f$ to $x$. It's not always the case that all the output edges of $x$ will lead to $f$, so we shouldn't just assume we have to consider all the output edges of $x$.

Keep these things in mind for the node implementations!

## Graphs, Nodes and Ops

I'd like to draw attention to a few things from the previous section.

1. The picture of the broken down expression and subexpressions of $f$ resembles a graph where the nodes are function applications.
2. We can use of dynamic programming to make computing backpropagation efficient. Even in our simple example we see the reuse of $\frac{\partial f}{\partial g}$ and $\frac{\partial f}{\partial h}$. As our graph grows in size and complexity, it becomes much more evident how wasteful it is to recompute partials. The cornerstone of dynamic programming is **solving a large problem through many smaller ones** and **caching**. We'll do both!

In the following exercises that you'll implement the forward and backward passes for the nodes in our graph. 

You'll write your code in `miniflow.py` (same directory), the autoreload extension will automatically reload your code when you make a change!

In [ ]:
%load_ext autoreload
%autoreload 2
from miniflow import *

### Exercise - Implement $f = (x * y) + (x * z)$ using nodes

Implement the `Mul` and `Add` nodes. The `Input` node is already provided.

Take a moment to look at the `miniflow.py` file and get familiar with the node classes, don't worry too much about the other functions.

In [ ]:
x = Input()
y = Input()
z = Input()

# TODO: implement Mul and Add in miniflow.py
g = Mul(x, y)
h = Mul(x, z)
f = Add(g, h)

# x, y, z nodes will take on these values and pass them to their outputs
feed_dict = {x: 3, y: 4, z: -5}
# compute the derivatives with respect to the following nodes
wrt = [x, y, z]

value, grad = value_and_grad(f, feed_dict, wrt)
assert value == -3 # should be -3
assert grad == [-1, 3, 3] # should be [-1, 3, 3]

## Functions for Neural Networks

We are now going to take our focus on how we can use differentiable graphs to compute functions for neural networks. 
Let's assume we have a vector of features $x$, a vector of weights $w$ and a bias scalar $b$. Then we to compute output we would perform a linear transform. It's recommend you use [numpy](http://www.numpy.org/) for the following exercises.

$$
o = (\sum_i x_iw_i) + b
$$

Or more concisely expressed as a dot product

$$
o =  x^Tw + b
$$

What if we have multiple outputs? Say we have $n$ features and $k$ outputs, then $b$ is a vector of length $k$, $x$ is a vector of length $n$ and $w$ becomes a $n$ by $k$ matrix, which we'll call $W$ from now on (matrices notation is typically a capital letter).

$$
o = x^TW + b
$$

What if we now have $m$ inputs? This is very common in practice feed in more than 1 input, it's referred to as the batch size. Then $x$ becomes a $m$ by $n$ matrix, we'll call this $X$.

$$
o = XW + b
$$

There we have it the famous linear transform! This on it's own though, is not all that powerful. We'll only do well if the data is linearably separable. This is where non-linear activations and layer stacking come into play. In fact, even a 2-layer neural network can [approximate arbitrary functions](http://neuralnetworksanddeeplearning.com/chap4.html). Pretty cool! There is however, a very fine line between being able to approximate any function theoretically and actually being able to do it efficiently and effectively in practice. If it was that easy then we wouldn't have convolution networks, recurrent neural networks, residual neural networks, generative neural network models, etc.

For this lab though, we'll keep it relatively simple. By the end of the lab you'll be able to construct a train a neural network with the following architecture.

$$
Input \rightarrow Linear \rightarrow Sigmoid \rightarrow Linear \rightarrow Softmax \rightarrow CrossEntropyLoss
$$

### Exercise - Implement `Linear` Node

In this exercise we'll implement the `Linear` node. This corresponds to the following function:

$$
f(X, W, b) = XW + b
$$

There are a few ways to go about the implementation, here are some possibilities:

1. Treat each element of the matrices and vector as a scalar and use existing `Mul` and `Add` nodes. You might have to implement a `Sum` node as well.
2. Break the function up into two subfunctions, you might call these `MatMul` and `BiasAdd`.
3. Treat it as one function.

Independent of the option chosen, the `dvalues` attribute of the `Linear` node should have 3 key, value pairs. Where the value is the same size/shape as the key. Example: If $W$ is of size $nxm$ then `dvalues[W]` should also be $nxm$.

**Tip: Write out the full above expression on paper and consider the derivative of $f$ with respect to a single element of $W, X, b$**

In [ ]:
x_in, w_in, b_in = Input(), Input(), Input()
# TODO: implement Linear
f = Linear(x_in, w_in, b_in)

x = np.array([[-1., -2.], [-1, -2]])
w = np.array([[2., -3], [2., -3]])
b = np.array([-3., -3]).reshape(1, -1)

feed_dict = {x_in: x, w_in: w, b_in: b}
loss, grad = value_and_grad(f, feed_dict, (x_in, w_in, b_in))
assert np.allclose(loss, np.array([[-9.,  6.], [-9.,  6.]]))
assert np.allclose(grad[0], np.array([[-1.,  -1.], [-1.,  -1.]]))
assert np.allclose(grad[1], np.array([[-2.,  -2.], [-4., -4.]]))
assert np.allclose(grad[2], np.array([[2., 2.]]))

### Exercise - Implement `Sigmoid` Node

In this exercise we'll implement the `Sigmoid` node. This corresponds to the [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) function:

$$
sigmoid(x) = \frac {1} {1 + exp(-x)}
$$

There are a 2 ways to go about the implementation:

1. Break it up into subfunctions, `Add`, `Divide`, `Exp`, etc.
2. Try to take the derivative of the sigmoid function and see if you can simplify it. The result is suprisingly simple!

In [ ]:
x_in = Input()
# TODO: implement Sigmoid
f = Sigmoid(x_in)

x = np.array([-10., 0, 10])
feed_dict = {x_in: x}
loss, grad = value_and_grad(f, feed_dict, [x_in])
assert np.allclose(loss, np.array([0., 0.5, 1.]), atol=1.e-4)
assert np.allclose(grad, np.array([0., 0.25, 0.]), atol=1.e-4)

### Exercise - Implement `CrossEntropyWithSoftmax` Node

In this exercise you'll implement the `CrossEntropyWithSoftmax` node. This corresponds to implementing the [softmax](https://en.wikipedia.org/wiki/Softmax_function) and [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) functions.

$$
softmax(x) = \frac{e^{x_i}} {\sum_j e^{x_j}}
$$

The input to the softmax function is a vector and the output is a vector of normalized probabilities. The input could also be a matrix in this case each row/example should be treated in isolation, output in this case would be a matrix of the same shape.

Example:

```
a = [0.2, 1.0, 0.3]
softmax(a) # [0.2309, 0.5138, 0.2551]
```

$$
cross\_entropy\_loss(probs, labels) = \frac {1} {n} \sum_{i=1}^n - log(probs[i, labels_i])
$$

Where $probs$ is the output of the $softmax$ function, $labels$ are the target labels (integer values from 0 to $k-1$) and $n$ is the number of rows of $probs$. For each row $i$ we pick the element at the index corresponding with the label.

Ok, but how will this loss encourage our model to classify correctly?

There are 2 key pieces of information here:

1. $probs$ contains values between 0 and 1
2. $log(0) = -inf$ and $log(1) = 0$; the possible log values are either negative or 0.

Clearly we'd like the probability of the index of the correct label to be 1 or close to it and if it's close to 0 we'll be heavily penalized and our will loss to shoot up. Note the negative sign sets the problem up nicely for minimizing the loss.

Once again, 2 ways to about the implementation:

1. Break it down into subfunctions.
2. Similar to the sigmoid function, we can simplify the derivative. The result is even simpler than sigmoid's!

In [ ]:
x_in = Input()
y_in = Input()
# TODO: implement CrossEntropyWithSoftmax
f = CrossEntropyWithSoftmax(x_in, y_in)

# pretend output of a softmax
x = np.array([[0.5, 1., 1.5]])
y = np.array([1])
feed_dict = {x_in: x, y_in: y}
loss, grad = value_and_grad(f, feed_dict, wrt=[x_in])
assert np.allclose(loss, 1.1802, atol=1.e-4)
assert np.allclose(grad, np.array([[0.1863, -0.6928,  0.5064]]), atol=1.e-4)

## Stochastic Gradient Descent (SGD)

At this point you should have all the nodes implemented and working correctly. Computing those weight gradients is a breeze! Now that we have the gradients what do we do with them?

As it turns out the gradient is analogous to the **steepest ascent direction**. Let's imagine that we are our neural network and we're in a park, not just any park though, a hilly park. We also know that we perform our best when we're on the highest hill, so let's get there shall we! There are a couple of issues though ... We are 1) blindfolded and 2) dropped randomly somewhere in the park (random weights!). It's easy to see we'd be hard pressed to find the highest hill in our current condition.

Luckily for us, we have a talking bird by our side, her name is Gradient and she's very good at informing us which direction causes us to ascend the hill the fastest. Unfortunately for Gradient, her vision isn't the greatest so it only applies to the local area. It might be the case we get to the top of a hill, but not the highest one.

Typically we're minimizing a function, not maximizing. The above analogy still holds, we're just descending instead of ascending (searching for lowest valley vs. highest hill). Making this adjustment is very simple, just put a negative sign on the gradient.

There's one more thing to think about, we know what direction to step in but how large of a step should we take? Too small a step and it could take a very long time to converge, and too large a step could overshoot the target cause divergence. In pratice the most common step sizes are 1e-3 and 1e-4, sometimes 1e-2 and 1e-1 work well too. Note: the *step size* is more commonly known as the *learning rate*.

Here's roughly what SGD looks like with miniflow.

```
f = ... # output node
wrt = [...] # weight input nodes
W = ... # weight for Linear node
b = ... # bias for Linear node
while True:
    feed_dict = {...} # new batch of data
    loss, grad = value_and_grad(f, feed_dict, wrt)
    # get the grad for W and b
    dW = grad[...]
    db = grad[...] 
    # descending in the direction of the gradient
    W[:] -= step_size * dW
    b[:] -= step_size * db
    
```

### Exercise: Train a neural net on MNIST with SGD

You now have all the pieces in place, all that's left to do is fit them together like legos.

TODO: lego picture

Your task is now to classify handwritten digits from the MNIST dataset.

In [ ]:
# load MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

In [ ]:
# getting input and label data
X_batch, y_batch = mnist.train.next_batch(128)
print(X_batch.shape, y_batch.shape)

In [ ]:
X_val = mnist.validation.images
y_val = mnist.validation.labels.astype(np.int8)
X_test = mnist.test.images
y_test = mnist.test.labels.astype(np.int8)

### Exercise: Train a neural net on MNIST with SGD

You should hit above 90% accuracy on even a 1-layer neural network.

In [ ]:
# TODO: initialize weights

# TODO: define architecture

In [ ]:
# TODO: set step size and batch size
step_size = ...
batch_size = ...
for i in range(10000):
    X_batch, y_batch = mnist.train.next_batch(batch_size)
    y_batch = y_batch.astype(np.int8)
    # TODO: get the loss and grad
    loss, grad = value_and_grad(...)
    
    if i % 1000 == 0:
        print("Iteration {}".format(i))
        print("Loss = {}".format(loss))
        # TODO: get the accuracy of the validation dataset
        # similar to value_and_grad except there's no wrt list
        acc = accuracy(...)
        print("Validation Accuracy = {}".format(acc))
        print('----------------')
        
    # TODO: SGD updates with gradient
   
    
# TODO: get the accuracy of the testing dataset
acc = accuracy(...)
print('----------------')
print("Testing Accuracy = {}".format(acc))

## Outro to TensorFlow

In this lab you created a library, miniflow, for computing differentiable graphs. Using miniflow, you created a neural network that can classiify handwritten digits with high accuracy. Not too shabby! Hopefully you now have a solid grasp of differentiable graphs and backpropagation. 

You probably also noticed how tedious all this work can be! It would be nice if someone already had written all this code for us, made it really fast, usable on GPUs, mobile devices and had a gigantic ecosystem around it so we can be assured it'll just get better and better ... Oh wait! It's not a fantasy. All of that is true and it's called [Tensorflow](https://www.tensorflow.org/), developed by the fine folks at Google and now a thriving open source project! From now on we'll use TensorFlow, or miniflow if you prefer, since you've grown so attached ...